In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/restaurant-revenue-prediction-dataset/restaurant_data.csv


In [2]:
dataset = pd.read_csv('/kaggle/input/restaurant-revenue-prediction-dataset/restaurant_data.csv')
# print(dataset)
df = dataset

In [3]:
# Drop kolom Name
df = df.drop(['Name'], axis=1)

# Cek hasilnya
print(df.shape)
print(df.head())

(8368, 16)
   Location   Cuisine  Rating  Seating Capacity  Average Meal Price  \
0     Rural  Japanese     4.0                38               73.98   
1  Downtown   Mexican     3.2                76               28.11   
2     Rural   Italian     4.7                48               48.29   
3     Rural   Italian     4.4                34               51.55   
4  Downtown  Japanese     4.9                88               75.98   

   Marketing Budget  Social Media Followers  Chef Experience Years  \
0              2224                   23406                     13   
1              4416                   42741                      8   
2              2796                   37285                     18   
3              1167                   15214                     13   
4              3639                   40171                      9   

   Number of Reviews  Avg Review Length  Ambience Score  \
0                185         161.924906             1.3   
1                533   

In [4]:
print(df.describe())

            Rating  Seating Capacity  Average Meal Price  Marketing Budget  \
count  8368.000000       8368.000000         8368.000000       8368.000000   
mean      4.008258         60.212835           47.896659       3218.254900   
std       0.581474         17.399488           14.336767       1824.896053   
min       3.000000         30.000000           25.000000        604.000000   
25%       3.500000         45.000000           35.490000       1889.000000   
50%       4.000000         60.000000           45.535000       2846.500000   
75%       4.500000         75.000000           60.300000       4008.500000   
max       5.000000         90.000000           76.000000       9978.000000   

       Social Media Followers  Chef Experience Years  Number of Reviews  \
count             8368.000000            8368.000000        8368.000000   
mean             36190.621773              10.051984         523.010397   
std              18630.153330               5.516606         277.215127 

In [5]:
print(df.isnull().sum())

Location                  0
Cuisine                   0
Rating                    0
Seating Capacity          0
Average Meal Price        0
Marketing Budget          0
Social Media Followers    0
Chef Experience Years     0
Number of Reviews         0
Avg Review Length         0
Ambience Score            0
Service Quality Score     0
Parking Availability      0
Weekend Reservations      0
Weekday Reservations      0
Revenue                   0
dtype: int64


In [6]:
print("Location:", df['Location'].unique())
print("Cuisine:", df['Cuisine'].unique())
print("Parking Availability:", df['Parking Availability'].unique())

Location: ['Rural' 'Downtown' 'Suburban']
Cuisine: ['Japanese' 'Mexican' 'Italian' 'Indian' 'French' 'American']
Parking Availability: ['Yes' 'No']


In [7]:
# Cara mudah dengan pandas
df = pd.get_dummies(df, columns=['Location', 'Cuisine', 'Parking Availability'], drop_first=True)

print(df.shape)
print(df.columns.tolist())

(8368, 21)
['Rating', 'Seating Capacity', 'Average Meal Price', 'Marketing Budget', 'Social Media Followers', 'Chef Experience Years', 'Number of Reviews', 'Avg Review Length', 'Ambience Score', 'Service Quality Score', 'Weekend Reservations', 'Weekday Reservations', 'Revenue', 'Location_Rural', 'Location_Suburban', 'Cuisine_French', 'Cuisine_Indian', 'Cuisine_Italian', 'Cuisine_Japanese', 'Cuisine_Mexican', 'Parking Availability_Yes']


In [8]:
X = df.drop(['Revenue'], axis=1)
y = df['Revenue']

print(X.shape)
print(y.shape)

(8368, 20)
(8368,)


In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print(X_train.shape)
print(X_test.shape)

(6694, 20)
(1674, 20)


In [10]:
from sklearn.linear_model import LinearRegression

# Buat model
regressor = LinearRegression()

# Train model
regressor.fit(X_train, y_train)

print("Model trained successfully!")

Model trained successfully!


In [11]:
y_pred = regressor.predict(X_test)

In [12]:
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.values.reshape(len(y_test),1)),1))

[[ 585595.02351082  517141.51      ]
 [ 463710.52800625  469501.35      ]
 [ 319926.81617713  342779.65      ]
 ...
 [ 890869.49489941  883718.71      ]
 [1034919.8567129  1077347.35      ]
 [ 819719.10346368  739529.05      ]]


In [13]:
from sklearn.metrics import r2_score
print("R² Score:", r2_score(y_test, y_pred))

R² Score: 0.9561391937272901


In [14]:
print("Intercept (b0):", regressor.intercept_)
print("Coefficients (b1, b2, ...):", regressor.coef_)

Intercept (b0): -651035.6839376037
Coefficients (b1, b2, ...): [ 1.39379076e+03  1.05970393e+04  1.34972406e+04 -2.39831095e+00
  4.46094320e-01  9.15482553e+02  1.17168286e+00 -8.81194795e-01
  2.02170819e+02  6.79115825e+01 -6.28504149e+00 -5.36120913e-01
  9.28370008e+02  2.09144220e+03 -4.34285901e+03  2.29791026e+03
 -2.22395721e+03 -1.05225903e+04  2.62216436e+03 -1.51055612e+03]


In [15]:
coef_df = pd.DataFrame({
    'Variable': X.columns,
    'Coefficient': regressor.coef_
})

coef_df = coef_df.reindex(coef_df['Coefficient'].abs().sort_values(ascending=False).index)
print(coef_df.to_string())

                    Variable   Coefficient
2         Average Meal Price  13497.240556
1           Seating Capacity  10597.039266
17          Cuisine_Japanese -10522.590280
14            Cuisine_French  -4342.859008
18           Cuisine_Mexican   2622.164360
15            Cuisine_Indian   2297.910260
16           Cuisine_Italian  -2223.957214
13         Location_Suburban   2091.442197
19  Parking Availability_Yes  -1510.556117
0                     Rating   1393.790758
12            Location_Rural    928.370008
5      Chef Experience Years    915.482553
8             Ambience Score    202.170819
9      Service Quality Score     67.911582
10      Weekend Reservations     -6.285041
3           Marketing Budget     -2.398311
6          Number of Reviews      1.171683
7          Avg Review Length     -0.881195
11      Weekday Reservations     -0.536121
4     Social Media Followers      0.446094
